In [ ]:
from azure.identity import DefaultAzureCredential,
InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
# Fall back to InteractiveBrowserCredential in case
    DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y =diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values,diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rateof', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train,y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
code="./src",
command="python diabetes-training.py",
environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
compute="aml-cluster",
display_name="diabetes-train-curated-env",
experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name)
    
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-
py37-cpu", version=44)
print(env. description, env.tags)
    

In [ ]:
from azure.ai.ml.entities import Environment
env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
    )
ml_client.environments.create_or_update(env_docker_image)

In [ ]:
from azure.ai.ml import command
# configure job
job = command(
code="./src",
command="python diabetes-training.py",
environment="docker-image-example:1",
compute="aml-cluster",
display_name="diabetes-train-custom-env",
experiment_name="diabetes-training"
)
# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
     - conda-forge
dependencies:
    - python=3.11
    - scikit-learn
    - pandas
    - numpy
    - matplotlib

In [ ]:
from azure.ai.ml.entities import Environment
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
        )
ml_client.environments.create_or_update(env_docker_conda)

In [ ]:
from azure.ai.ml import command
# configure job
job = command(
code="./src",
command="python diabetes-training.py",
environment="docker-image-plus-conda-example:1",
compute="aml-cluster",
display_name="diabetes-train-custom-env",
experiment_name="diabetes-training"